In [ ]:
# ============================================================================
# MULTI-ASSET FINANCIAL SENTIMENT ANALYZER - SINGLE FILE VERSION
# ============================================================================

# 1. INSTALL ALL REQUIRED PACKAGES
!pip install pandas numpy requests beautifulsoup4 tweepy textblob nltk transformers torch plotly dash dash-bootstrap-components yfinance alpha_vantage sqlalchemy schedule python-dotenv -q
!pip install nest_asyncio pyngrok -q  


In [2]:
# For Colab compatibility

import nest_asyncio
nest_asyncio.apply()

# 2. IMPORT ALL LIBRARIES
import os
import sys
import json
import warnings
import argparse
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass
from abc import ABC, abstractmethod
from collections import defaultdict
import time
import random # Added: Import random module

# Data manipulation
import pandas as pd
import numpy as np

# Data collection
import requests
from bs4 import BeautifulSoup
import tweepy
import yfinance as yf
from alpha_vantage.alphavantage import AlphaVantage

# NLP & Sentiment Analysis
import nltk
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
# from jupyter_dash import JupyterDash  # For Colab # Removed jupyter_dash

# Database & Utilities
import sqlite3
import pickle
import schedule
from IPython.display import display, clear_output

warnings.filterwarnings('ignore')

# Download NLTK data
try:
    nltk.download('vader_lexicon', quiet=True)
    nltk.download('punkt', quiet=True)
except:
    pass

d:\self made projects\Sentiment_analysis\.venv-2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ============================================================================
# CONFIGURATION
# ============================================================================

class Config:
    """Configuration manager for the application"""

    # API Keys (Use free tier or demo keys - replace with your own)
    NEWS_API_KEY = 'YOUR_NEWS_API_KEY'  # Get from https://newsapi.org
    TWITTER_BEARER_TOKEN = 'YOUR_TWITTER_TOKEN'  # Twitter Developer Portal
    ALPHA_VANTAGE_KEY = 'demo'  # Use 'demo' for testing or get from https://www.alphavantage.co

    # Asset Classes Configuration
    ASSET_CLASSES = {
        'equity': {
            'US': ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'SPY', 'QQQ'],
            'UK': ['TSCO.L', 'BP.L', 'HSBA.L', 'VOD.L', 'LLOY.L', 'FTSE'],
            'INDIA': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'SBIN.NS', 'NIFTY']
        },
        'commodities': {
            'METALS': ['GC=F', 'SI=F', 'PL=F', 'PA=F'],
            'ENERGY': ['CL=F', 'BZ=F', 'NG=F', 'HO=F'],
            'AGRICULTURE': ['ZC=F', 'ZW=F', 'ZS=F', 'KC=F']
        },
        'forex': {
            'MAJORS': ['EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X'],
            'MINORS': ['EURGBP=X', 'GBPJPY=X', 'AUDUSD=X', 'USDCAD=X'],
            'EXOTIC': ['USDINR=X', 'USDMXN=X', 'USDZAR=X']
        },
        'crypto': {
            'MAJOR': ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD'],
            'ALTCOINS': ['ADA-USD', 'SOL-USD', 'DOGE-USD', 'DOT-USD']
        }
    }

    # Sentiment thresholds
    SENTIMENT_THRESHOLDS = {
        'very_negative': -0.75,
        'negative': -0.25,
        'neutral': 0.25,
        'positive': 0.75,
        'very_positive': 1.0
    }

    # Colors for sentiment
    SENTIMENT_COLORS = {
        'Very Negative': '#FF0000',
        'Negative': '#FF6666',
        'Neutral': '#CCCCCC',
        'Positive': '#66FF66',
        'Very Positive': '#00AA00'
    }

    @staticmethod
    def get_all_tickers():
        """Get all unique tickers from configuration"""
        tickers = []
        for asset_class in Config.ASSET_CLASSES.values():
            for market in asset_class.values():
                tickers.extend(market)
        return list(set(tickers))

In [4]:
# ============================================================================
# DATA COLLECTORS
# ============================================================================

class NewsCollector:
    """Collect news from various sources"""

    def __init__(self, api_key=None):
        self.api_key = api_key or Config.NEWS_API_KEY
        self.base_url = "https://newsapi.org/v2/everything"

    def fetch_news(self, ticker, days_back=7, max_results=20):
        """Fetch news articles for a ticker"""
        if self.api_key == 'YOUR_NEWS_API_KEY':
            # Return dummy data for demo
            return self._get_dummy_news(ticker)

        from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')

        params = {
            'q': ticker,
            'from': from_date,
            'sortBy': 'publishedAt',
            'language': 'en',
            'apiKey': self.api_key,
            'pageSize': min(max_results, 100)
        }

        try:
            response = requests.get(self.base_url, params=params, timeout=10)
            if response.status_code == 200:
                return self._parse_news_response(response.json())
        except Exception as e:
            print(f"Error fetching news: {e}")

        return self._get_dummy_news(ticker)

    def _parse_news_response(self, data):
        """Parse news API response"""
        articles = []
        for article in data.get('articles', []):
            articles.append({
                'source': article.get('source', {}).get('name', 'Unknown'),
                'title': article.get('title', 'No title'),
                'description': article.get('description', ''),
                'url': article.get('url', '#'),
                'published_at': article.get('publishedAt', ''),
                'content': article.get('content', '')
            })
        return articles[:20]  # Limit to 20 articles

    def _get_dummy_news(self, ticker):
        """Generate dummy news for demo purposes"""
        import random
        sources = ['Bloomberg', 'Reuters', 'Financial Times', 'CNBC', 'Wall Street Journal']
        sentiments = ['positive', 'negative', 'neutral']

        articles = []
        for i in range(10):
            sentiment = random.choice(sentiments)
            if sentiment == 'positive':
                title = f"{ticker} shows strong growth potential in Q4"
                desc = f"Analysts are bullish on {ticker} as earnings exceed expectations."
            elif sentiment == 'negative':
                title = f"Concerns rise over {ticker}'s market position"
                desc = f"Recent reports suggest challenges ahead for {ticker}."
            else:
                title = f"{ticker} maintains steady performance"
                desc = f"{ticker} shows mixed signals in recent trading sessions."

            articles.append({
                'source': random.choice(sources),
                'title': title,
                'description': desc,
                'url': f'https://example.com/news/{ticker}-{i}',
                'published_at': (datetime.now() - timedelta(days=random.randint(0, 7))).isoformat(),
                'content': desc
            })

        return articles

class TwitterCollector:
    """Collect tweets for sentiment analysis"""

    def __init__(self, bearer_token=None):
        self.bearer_token = bearer_token or Config.TWITTER_BEARER_TOKEN

    def fetch_tweets(self, ticker, max_results=50):
        """Fetch tweets mentioning a ticker"""
        if self.bearer_token == 'YOUR_TWITTER_TOKEN':
            # Return dummy tweets for demo
            return self._get_dummy_tweets(ticker)

        try:
            client = tweepy.Client(bearer_token=self.bearer_token)
            query = f"${ticker} OR #{ticker} lang:en -is:retweet"

            tweets = client.search_recent_tweets(
                query=query,
                max_results=min(max_results, 100),
                tweet_fields=['created_at', 'public_metrics', 'text'],
                user_fields=['username']
            )

            if tweets.data:
                return self._parse_tweets(tweets)
        except Exception as e:
            print(f"Error fetching tweets: {e}")

        return self._get_dummy_tweets(ticker)

    def _parse_tweets(self, tweets_data):
        """Parse tweet data"""
        tweets = []
        for tweet in tweets_data.data:
            tweets.append({
                'text': tweet.text,
                'created_at': tweet.created_at,
                'retweet_count': tweet.public_metrics['retweet_count'],
                'like_count': tweet.public_metrics['like_count'],
                'reply_count': tweet.public_metrics['reply_count']
            })
        return tweets

    def _get_dummy_tweets(self, ticker):
        """Generate dummy tweets for demo"""
        import random
        sentiments = ['positive', 'negative', 'neutral']

        tweets = []
        for i in range(20):
            sentiment = random.choice(sentiments)
            if sentiment == 'positive':
                text = f"Bullish on ${ticker}! Strong fundamentals and great management. #investing"
            elif sentiment == 'negative':
                text = f"Concerned about ${ticker} recent performance. Might be time to reconsider. #stocks"
            else:
                text = f"${ticker} showing mixed signals today. Waiting for more data. #trading"

            tweets.append({
                'text': text,
                'created_at': datetime.now() - timedelta(hours=random.randint(0, 48)),
                'retweet_count': random.randint(0, 100),
                'like_count': random.randint(0, 500),
                'reply_count': random.randint(0, 50)
            })

        return tweets

class PriceDataCollector:
    """Collect price data from Yahoo Finance"""

    def __init__(self):
        self.cache = {}

    def get_price_data(self, ticker, period="1mo"):
        """Get price history for a ticker"""
        cache_key = f"{ticker}_{period}"

        if cache_key in self.cache:
            if (datetime.now() - self.cache[cache_key]['timestamp']).seconds < 300:  # 5 min cache
                return self.cache[cache_key]['data']

        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period=period)

            if not hist.empty:
                # Calculate additional metrics
                hist['Returns'] = hist['Close'].pct_change()
                hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
                hist['Volume_SMA'] = hist['Volume'].rolling(window=10).mean()

                self.cache[cache_key] = {
                    'data': hist,
                    'timestamp': datetime.now()
                }

                return hist
        except Exception as e:
            print(f"Error fetching price data for {ticker}: {e}")

        # Return dummy data if API fails
        return self._generate_dummy_price_data(ticker, period)

    def get_current_price(self, ticker):
        """Get current price for a ticker"""
        try:
            stock = yf.Ticker(ticker)
            data = stock.history(period='1d')
            if not data.empty:
                return data['Close'].iloc[-1]
        except:
            pass
        return np.random.uniform(50, 500)  # Dummy price

    def _generate_dummy_price_data(self, ticker, period):
        """Generate dummy price data for demo"""
        dates = pd.date_range(end=datetime.now(), periods=30, freq='D')

        # Start with random price
        base_price = np.random.uniform(50, 500)
        prices = []

        for i in range(30):
            # Random walk with slight upward bias
            change = np.random.normal(0.001, 0.02)
            base_price *= (1 + change)
            prices.append(base_price)

        df = pd.DataFrame({
            'Open': [p * (1 + np.random.uniform(-0.01, 0.01)) for p in prices],
            'High': [p * (1 + np.random.uniform(0, 0.03)) for p in prices],
            'Low': [p * (1 - np.random.uniform(0, 0.03)) for p in prices],
            'Close': prices,
            'Volume': [np.random.randint(1000000, 10000000) for _ in range(30)]
        }, index=dates)

        df['Returns'] = df['Close'].pct_change()
        df['SMA_20'] = df['Close'].rolling(window=20).mean()

        return df

In [5]:
# ============================================================================
# SENTIMENT ANALYSIS ENGINE
# ============================================================================

class SentimentAnalyzer:
    """Advanced sentiment analysis engine"""

    def __init__(self):
        # Initialize VADER
        self.vader = SentimentIntensityAnalyzer()

        # Initialize TextBlob
        self.textblob = TextBlob

        # Try to load FinBERT (optional)
        self.finbert = None
        try:
            self.finbert = pipeline(
                "sentiment-analysis",
                model="ProsusAI/finbert",
                device=-1  # Use CPU in Colab
            )
            print("✓ FinBERT model loaded successfully")
        except Exception as e:
            print(f"Note: FinBERT not available, using VADER and TextBlob: {e}")

        # Financial sentiment lexicon
        self.financial_lexicon = {
            'positive': ['bullish', 'growth', 'profit', 'gain', 'surge', 'rally', 'outperform',
                        'buy', 'strong', 'positive', 'optimistic', 'beat', 'exceed'],
            'negative': ['bearish', 'loss', 'decline', 'drop', 'fall', 'plunge', 'underperform',
                        'sell', 'weak', 'negative', 'pessimistic', 'miss', 'warning', 'risk']
        }

    def analyze_text(self, text):
        """Analyze sentiment of a single text"""
        if not text or len(str(text)) < 10:
            return self._get_neutral_sentiment()

        text = str(text)

        # 1. VADER sentiment
        vader_scores = self.vader.polarity_scores(text)
        vader_score = vader_scores['compound']

        # 2. TextBlob sentiment
        try:
            blob = self.textblob(text)
            textblob_score = blob.sentiment.polarity
        except:
            textblob_score = 0

        # 3. FinBERT if available
        finbert_score = 0
        if self.finbert:
            try:
                result = self.finbert(text[:512])[0]
                if result['label'] == 'positive':
                    finbert_score = result['score']
                elif result['label'] == 'negative':
                    finbert_score = -result['score']
            except:
                pass

        # 4. Financial lexicon check
        lexicon_score = self._check_financial_lexicon(text)

        # Combine scores (weighted average)
        scores = [vader_score, textblob_score, finbert_score, lexicon_score]
        weights = [0.3, 0.3, 0.3, 0.1]

        # Remove None values and adjust weights
        valid_scores = [s for s in scores if s is not None]
        if valid_scores:
            valid_weights = weights[:len(valid_scores)]
            valid_weights = [w/sum(valid_weights) for w in valid_weights]
            composite_score = sum(w * s for w, s in zip(valid_weights, valid_scores))
        else:
            composite_score = 0

        # Classify sentiment
        sentiment_label = self._classify_sentiment(composite_score)

        return {
            'composite_score': composite_score,
            'vader_score': vader_score,
            'textblob_score': textblob_score,
            'finbert_score': finbert_score,
            'lexicon_score': lexicon_score,
            'sentiment_label': sentiment_label,
            'confidence': abs(composite_score)
        }

    def _check_financial_lexicon(self, text):
        """Check for financial terms in text"""
        text_lower = text.lower()
        positive_count = sum(1 for word in self.financial_lexicon['positive'] if word in text_lower)
        negative_count = sum(1 for word in self.financial_lexicon['negative'] if word in text_lower)

        if positive_count == 0 and negative_count == 0:
            return 0

        return (positive_count - negative_count) / (positive_count + negative_count)

    def _classify_sentiment(self, score):
        """Classify sentiment based on score"""
        thresholds = Config.SENTIMENT_THRESHOLDS

        if score <= thresholds['very_negative']:
            return 'Very Negative'
        elif score <= thresholds['negative']:
            return 'Negative'
        elif score <= thresholds['neutral']:
            return 'Neutral'
        elif score <= thresholds['positive']:
            return 'Positive'
        else:
            return 'Very Positive'

    def _get_neutral_sentiment(self):
        """Return neutral sentiment for empty/invalid text"""
        return {
            'composite_score': 0,
            'vader_score': 0,
            'textblob_score': 0,
            'finbert_score': 0,
            'lexicon_score': 0,
            'sentiment_label': 'Neutral',
            'confidence': 0
        }

    def analyze_batch(self, texts):
        """Analyze multiple texts"""
        results = []
        for text in texts:
            results.append(self.analyze_text(text))
        return pd.DataFrame(results)

    def calculate_asset_sentiment(self, news_articles, tweets):
        """Calculate overall sentiment for an asset"""
        all_texts = []

        # Extract text from news
        for article in news_articles:
            all_texts.append(article.get('title', ''))
            all_texts.append(article.get('description', ''))

        # Extract text from tweets
        for tweet in tweets:
            all_texts.append(tweet.get('text', ''))

        if not all_texts:
            return self._get_default_sentiment()

        # Analyze all texts
        sentiment_df = self.analyze_batch(all_texts)

        # Calculate statistics
        overall_score = sentiment_df['composite_score'].mean()
        sentiment_label = self._classify_sentiment(overall_score)

        # Count by sentiment
        positive_count = len(sentiment_df[sentiment_df['composite_score'] > 0.25])
        negative_count = len(sentiment_df[sentiment_df['composite_score'] < -0.25])
        neutral_count = len(sentiment_df) - positive_count - negative_count

        # Calculate confidence (higher when agreement is strong)
        confidence = sentiment_df['composite_score'].std()
        if confidence == 0:
            confidence = 1  # Perfect agreement

        return {
            'overall_score': overall_score,
            'sentiment_label': sentiment_label,
            'sources_analyzed': len(all_texts),
            'positive_count': positive_count,
            'negative_count': negative_count,
            'neutral_count': neutral_count,
            'confidence': 1 / (confidence + 0.1),  # Inverse of std dev
            'sentiment_distribution': {
                'positive': 0,
                'negative': 0,
                'neutral': 0
            },
            'detailed_scores': {
                'vader_avg': sentiment_df['vader_score'].mean(),
                'textblob_avg': sentiment_df['textblob_score'].mean(),
                'finbert_avg': sentiment_df['finbert_score'].mean()
            }
        }

    def _get_default_sentiment(self):
        """Return default sentiment when no data is available"""
        return {
            'overall_score': 0,
            'sentiment_label': 'Neutral',
            'sources_analyzed': 0,
            'positive_count': 0,
            'negative_count': 0,
            'neutral_count': 0,
            'confidence': 0,
            'sentiment_distribution': {
                'positive': 0,
                'negative': 0,
                'neutral': 0
            },
        }

In [6]:
# ============================================================================
# DATA STORAGE
# ============================================================================

class DataStorage:
    """Simple data storage using SQLite"""

    def __init__(self, db_path='sentiment_data.db'):
        self.db_path = db_path
        self._init_database()

    def _init_database(self):
        """Initialize database tables"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Create tables
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS asset_sentiment (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                timestamp DATETIME,
                sentiment_score REAL,
                sentiment_label TEXT,
                sources_analyzed INTEGER,
                price REAL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_sentiment (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                source TEXT,
                title TEXT,
                url TEXT,
                sentiment_score REAL,
                sentiment_label TEXT,
                timestamp DATETIME
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS price_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                date DATE,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER
            )
        ''')

        conn.commit()
        conn.close()

    def save_sentiment(self, ticker, sentiment_data, price=None):
        """Save sentiment analysis results"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO asset_sentiment
            (ticker, timestamp, sentiment_score, sentiment_label, sources_analyzed, price)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            ticker,
            datetime.now(),
            sentiment_data['overall_score'],
            sentiment_data['sentiment_label'],
            sentiment_data['sources_analyzed'],
            price
        ))

        conn.commit()
        conn.close()

    def save_news(self, ticker, news_articles, analyzer):
        """Save news articles with sentiment"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        for article in news_articles:
            sentiment = analyzer.analyze_text(
                f"{article.get('title', '')} {article.get('description', '')}"
            )

            cursor.execute('''
                INSERT INTO news_sentiment
                (ticker, source, title, url, sentiment_score, sentiment_label, timestamp)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                ticker,
                article.get('source', 'Unknown'),
                article.get('title', '')[:200],
                article.get('url', ''),
                sentiment['composite_score'],
                sentiment['sentiment_label'],
                datetime.now()
            ))

        conn.commit()
        conn.close()

    def get_sentiment_history(self, ticker, days=30):
        """Get sentiment history for a ticker"""
        conn = sqlite3.connect(self.db_path)
        query = '''
            SELECT * FROM asset_sentiment
            WHERE ticker = ? AND timestamp >= datetime('now', ?)
            ORDER BY timestamp DESC
        '''
        df = pd.read_sql_query(query, conn, params=[ticker, f'-{days} days'])
        conn.close()
        return df

    def get_recent_news(self, ticker, limit=10):
        """Get recent news for a ticker"""
        conn = sqlite3.connect(self.db_path)
        query = '''
            SELECT * FROM news_sentiment
            WHERE ticker = ?
            ORDER BY timestamp DESC
            LIMIT ?
        '''
        df = pd.read_sql_query(query, conn, params=[ticker, limit])
        conn.close()
        return df

In [7]:
# ============================================================================
# VISUALIZATION COMPONENTS
# ============================================================================

class Visualizer:
    """Create visualization components"""

    @staticmethod
    def create_sentiment_gauge(score, label, ticker):
        """Create a speedometer gauge for sentiment"""
        fig = go.Figure(go.Indicator(
            mode="gauge+number+delta",
            value=score,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={
                'text': f"{ticker}<br>Sentiment",
                'font': {'size': 20, 'color': 'white'}
            },
            delta={'reference': 0, 'font': {'size': 16}},
            gauge={
                'axis': {
                    'range': [-1, 1],
                    'tickwidth': 1,
                    'tickcolor': "white",
                    'tickfont': {'color': 'white'}
                },
                'bar': {'color': Config.SENTIMENT_COLORS.get(label, '#CCCCCC')},
                'bgcolor': "rgba(0,0,0,0)",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': [-1, -0.5], 'color': '#FF0000'},
                    {'range': [-0.5, -0.2], 'color': '#FF6666'},
                    {'range': [-0.2, 0.2], 'color': '#CCCCCC'},
                    {'range': [0.2, 0.5], 'color': '#66FF66'},
                    {'range': [0.5, 1], 'color': '#00AA00'}
                ],
                'threshold': {
                    'line': {'color': "white", 'width': 3},
                    'thickness': 0.8,
                    'value': score
                }
            }
        ))

        fig.update_layout(
            height=300,
            margin=dict(l=20, r=20, t=50, b=20),
            paper_bgcolor='rgba(0,0,0,0)',
            font={'color': "white", 'family': "Arial"}
        )

        return fig

    @staticmethod
    def create_price_chart(price_data, ticker):
        """Create candlestick price chart"""
        if price_data.empty:
            return go.Figure()

        fig = make_subplots(
            rows=2, cols=1,
            row_heights=[0.7, 0.3],
            vertical_spacing=0.05,
            subplot_titles=(f"{ticker} Price Chart", "Volume"),
            shared_xaxes=True
        )

        # Candlestick chart
        fig.add_trace(
            go.Candlestick(
                x=price_data.index,
                open=price_data['Open'],
                high=price_data['High'],
                low=price_data['Low'],
                close=price_data['Close'],
                name="Price",
                increasing_line_color='#00AA00',
                decreasing_line_color='#FF0000'
            ),
            row=1, col=1
        )

        # Add 20-day SMA
        if 'SMA_20' in price_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=price_data.index,
                    y=price_data['SMA_20'],
                    name="20-Day SMA",
                    line=dict(color='#FFA500', width=2)
                ),
                row=1, col=1
            )

        # Volume chart
        colors = ['#FF0000' if price_data['Close'].iloc[i] < price_data['Open'].iloc[i]
                 else '#00AA00' for i in range(len(price_data))]

        fig.add_trace(
            go.Bar(
                x=price_data.index,
                y=price_data['Volume'],
                name="Volume",
                marker_color=colors,
                opacity=0.7
            ),
            row=2, col=1
        )

        # Update layout
        fig.update_layout(
            height=600,
            showlegend=True,
            xaxis_rangeslider_visible=False,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font={'color': "white", 'family': "Arial"},
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')

        return fig

    @staticmethod
    def create_sentiment_trend(sentiment_history):
        """Create sentiment trend chart"""
        if sentiment_history.empty:
            return go.Figure()

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=sentiment_history['timestamp'],
            y=sentiment_history['sentiment_score'],
            mode='lines+markers',
            name='Sentiment Score',
            line=dict(color='#4A90E2', width=3),
            marker=dict(size=8)
        ))

        # Add zero line
        fig.add_shape(
            type="line",
            x0=sentiment_history['timestamp'].min(),
            y0=0,
            x1=sentiment_history['timestamp'].max(),
            y1=0,
            line=dict(color="white", width=2, dash="dash")
        )

        fig.update_layout(
            title="Sentiment Trend Over Time",
            height=400,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font={'color': "white", 'family': "Arial"},
            xaxis_title="Date",
            yaxis_title="Sentiment Score"
        )

        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')

        return fig

    @staticmethod
    def create_sentiment_distribution(sentiment_data):
        """Create sentiment distribution pie chart"""
        labels = ['Positive', 'Neutral', 'Negative']
        values = [
            sentiment_data['positive_count'],
            sentiment_data['neutral_count'],
            sentiment_data['negative_count']
        ]

        colors = ['#00AA00', '#CCCCCC', '#FF0000']

        fig = go.Figure(data=[go.Pie(
            labels=labels,
            values=values,
            hole=.3,
            marker_colors=colors,
            textinfo='label+percent',
            textfont={'color': 'white', 'size': 14}
        )])

        fig.update_layout(
            title="Sentiment Distribution",
            height=400,
            paper_bgcolor='rgba(0,0,0,0)',
            font={'color': "white", 'family': "Arial"},
            legend=dict(
                font=dict(color='white')
            )
        )

        return fig

In [8]:
# ============================================================================
# MAIN APPLICATION
# ============================================================================

class FinancialSentimentAnalyzer:
    """Main application class"""

    def __init__(self):
        # Initialize components
        self.config = Config()
        self.news_collector = NewsCollector()
        self.twitter_collector = TwitterCollector()
        self.price_collector = PriceDataCollector()
        self.sentiment_analyzer = SentimentAnalyzer()
        self.data_storage = DataStorage()
        self.visualizer = Visualizer()

        # Cache for performance
        self.cache = {
            'sentiment': {},
            'price_data': {},
            'news': {}
        }

        # State
        self.current_ticker = None
        self.current_data = None

        print("✓ Financial Sentiment Analyzer initialized")
        print(f"✓ Tracking {len(self.config.get_all_tickers())} assets across 4 markets")

    def analyze_asset(self, ticker):
        """Analyze a single asset"""
        print(f"🔍 Analyzing {ticker}...")

        # Clear cache for this ticker
        self.cache['sentiment'].pop(ticker, None)
        self.cache['price_data'].pop(ticker, None)
        self.cache['news'].pop(ticker, None)

        try:
            # Collect data in parallel (simulated)
            news = self.news_collector.fetch_news(ticker)
            tweets = self.twitter_collector.fetch_tweets(ticker)
            price_data = self.price_collector.get_price_data(ticker)
            current_price = self.price_collector.get_current_price(ticker)

            # Analyze sentiment
            sentiment = self.sentiment_analyzer.calculate_asset_sentiment(news, tweets)

            # Store results
            self.data_storage.save_sentiment(ticker, sentiment, current_price)
            self.data_storage.save_news(ticker, news, self.sentiment_analyzer)

            # Cache results
            self.current_ticker = ticker
            self.current_data = {
                'ticker': ticker,
                'sentiment': sentiment,
                'price_data': price_data,
                'news': news,
                'tweets': tweets,
                'current_price': current_price,
                'timestamp': datetime.now()
            }

            # Cache for future use
            self.cache['sentiment'][ticker] = sentiment
            self.cache['price_data'][ticker] = price_data
            self.cache['news'][ticker] = news

            print(f"✓ Analysis complete for {ticker}")
            print(f"  Sentiment: {sentiment['sentiment_label']} ({sentiment['overall_score']:.3f})")
            print(f"  Sources analyzed: {sentiment['sources_analyzed']}")
            print(f"  Current price: ${current_price:.2f}" if current_price else "  Price: N/A")

            return self.current_data

        except Exception as e:
            print(f"❌ Error analyzing {ticker}: {e}")
            return None

    def get_analysis_results(self, ticker=None):
        """Get analysis results for display"""
        if ticker and ticker != self.current_ticker:
            self.analyze_asset(ticker)

        if self.current_data is None:
            return None

        return {
            'ticker': self.current_data['ticker'],
            'sentiment': self.current_data['sentiment'],
            'price_data': self.current_data['price_data'],
            'news': self.current_data['news'][:10],  # Top 10 news
            'current_price': self.current_data['current_price'],
            'timestamp': self.current_data['timestamp']
        }

    def get_sentiment_history(self, ticker, days=30):
        """Get sentiment history for a ticker"""
        return self.data_storage.get_sentiment_history(ticker, days)

    def get_recent_news(self, ticker, limit=10):
        """Get recent news for a ticker"""
        return self.data_storage.get_recent_news(ticker, limit)

    def batch_analyze(self, tickers=None):
        """Analyze multiple tickers"""
        if tickers is None:
            tickers = self.config.get_all_tickers()[:10]  # Limit to 10 for demo

        results = []
        for ticker in tickers:
            try:
                result = self.analyze_asset(ticker)
                if result:
                    results.append({
                        'ticker': ticker,
                        'sentiment_score': result['sentiment']['overall_score'],
                        'sentiment_label': result['sentiment']['sentiment_label'],
                        'price': result.get('current_price', 0)
                    })
            except Exception as e:
                print(f"Skipping {ticker}: {e}")

        return pd.DataFrame(results)

    def generate_report(self, results_df):
        """Generate a summary report"""
        if results_df.empty:
            return "No data available for report."

        report = [
            "=" * 70,
            "FINANCIAL SENTIMENT ANALYSIS REPORT",
            f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            "=" * 70,
            f"\nTotal Assets Analyzed: {len(results_df)}",
            f"Average Sentiment Score: {results_df['sentiment_score'].mean():.3f}",
            f"Most Positive: {results_df.loc[results_df['sentiment_score'].idxmax()]['ticker']} "
            f"({results_df['sentiment_score'].max():.3f})",
            f"Most Negative: {results_df.loc[results_df['sentiment_score'].idxmin()]['ticker']} "
            f"({results_df['sentiment_score'].min():.3f})",
            "\nTop 5 by Sentiment:",
            "-" * 70
        ]

        top_positive = results_df.nlargest(5, 'sentiment_score')
        for _, row in top_positive.iterrows():
            report.append(
                f"{row['ticker']:15} | "
                f"Score: {row['sentiment_score']:7.3f} | "
                f"Sentiment: {row['sentiment_label']:15} | "
                f"Price: ${row['price']:.2f}"
            )

        report.append("\n" + "=" * 70)
        return "\n".join(report)

In [9]:
# ============================================================================
# DASHBOARD APPLICATION
# ============================================================================

class FinancialDashboard:
    """Interactive dashboard for sentiment analysis"""

    def __init__(self, analyzer):
        self.analyzer = analyzer

        # Initialize Dash app
        self.app = dash.Dash( # Changed from JupyterDash
            __name__,
            external_stylesheets=[dbc.themes.DARKLY],
            meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
        )

        self._setup_layout()
        self._setup_callbacks()

        print("✓ Dashboard initialized")

    def _setup_layout(self):
        """Setup dashboard layout"""
        # Header
        header = dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H1("📈 Multi-Asset Sentiment Analyzer",
                           className="text-center mt-3 mb-2"),
                    html.P("Advanced sentiment analysis across Equity, Commodities, Forex, and Cryptocurrency markets",
                          className="text-center text-light mb-4"),
                    html.Hr(className="my-2")
                ], width=12)
            ])
        ], fluid=True)

        # Controls panel
        controls = dbc.Card([
            dbc.CardBody([
                html.H5("🎯 Asset Selection", className="card-title mb-3"),

                # Asset class selection
                html.Label("Asset Class:", className="mb-1"),
                dcc.Dropdown(
                    id='asset-class-dropdown',
                    options=[
                        {'label': '📊 Equity Markets', 'value': 'equity'},
                        {'label': '⛏️ Commodities', 'value': 'commodities'},
                        {'label': '💱 Forex', 'value': 'forex'},
                        {'label': '₿ Cryptocurrency', 'value': 'crypto'}
                    ],
                    value='equity',
                    clearable=False,
                    className="mb-3"
                ),

                # Market selection
                html.Label("Market:", className="mb-1"),
                dcc.Dropdown(
                    id='market-dropdown',
                    clearable=False,
                    className="mb-3"
                ),

                # Ticker selection
                html.Label("Ticker:", className="mb-1"),
                dcc.Dropdown(
                    id='ticker-dropdown',
                    clearable=False,
                    className="mb-4"
                ),

                # Analyze button
                dbc.Button(
                    "🚀 Analyze Now",
                    id='analyze-button',
                    color="primary",
                    className="w-100 mb-3",
                    n_clicks=0
                ),

                # Refresh button
                dbc.Button(
                    "🔄 Refresh Data",
                    id='refresh-button',
                    color="secondary",
                    className="w-100",
                    n_clicks=0
                ),

                html.Hr(className="my-3"),

                # Quick stats
                html.H6("📊 Quick Stats", className="mt-3"),
                html.Div(id='quick-stats', className="small")
            ])
        ], className="h-100")

        # Main content area
        main_content = dbc.Tabs([
            # Tab 1: Overview
            dbc.Tab([
                dbc.Row([
                    dbc.Col([
                        html.Div(id='sentiment-gauge-container'),
                    ], width=4),
                    dbc.Col([
                        html.Div(id='price-chart-container'),
                    ], width=8)
                ], className="mb-4"),

                dbc.Row([
                    dbc.Col([
                        html.Div(id='sentiment-trend-container'),
                    ], width=6),
                    dbc.Col([
                        html.Div(id='sentiment-distribution-container'),
                    ], width=6)
                ])
            ], label="📈 Overview", tab_id="overview"),

            # Tab 2: News & Social
            dbc.Tab([
                dbc.Row([
                    dbc.Col([
                        html.H4("📰 Latest News", className="mb-3"),
                        html.Div(id='news-table-container')
                    ], width=12)
                ])
            ], label="📰 News & Social", tab_id="news"),

            # Tab 3: Market Overview
            dbc.Tab([
                dbc.Row([
                    dbc.Col([
                        html.H4("🌐 Market Overview", className="mb-3"),
                        dbc.Button(
                            "Run Market Analysis",
                            id='market-analysis-button',
                            color="info",
                            className="mb-3"
                        ),
                        html.Div(id='market-analysis-results')
                    ], width=12)
                ])
            ], label="🌐 Market Overview", tab_id="market"),

            # Tab 4: About
            dbc.Tab([
                dbc.Row([
                    dbc.Col([
                        html.H4("ℹ️ About This Tool", className="mb-3"),
                        html.P("""
                            This Financial Sentiment Analyzer is a professional-grade tool that:
                            • Analyzes sentiment across multiple asset classes
                            • Uses advanced NLP models (VADER, TextBlob, FinBERT)
                            • Collects data from news, social media, and financial APIs
                            • Provides real-time visualization and insights
                        """, className="lead"),
                        html.H5("Asset Classes Covered:", className="mt-4"),
                        html.Ul([
                            html.Li("Equity Markets: US, UK, India"),
                            html.Li("Commodities: Metals, Energy, Agriculture"),
                            html.Li("Forex: Majors, Minors, Exotics"),
                            html.Li("Cryptocurrency: Major coins & Altcoins")
                        ]),
                        html.H5("Data Sources:", className="mt-4"),
                        html.Ul([
                            html.Li("News: NewsAPI, Financial Publications"),
                            html.Li("Social Media: Twitter/X"),
                            html.Li("Price Data: Yahoo Finance, Alpha Vantage"),
                            html.Li("Sentiment Analysis: Custom NLP pipeline")
                        ])
                    ], width=12)
                ])
            ], label="ℹ️ About", tab_id="about")
        ])

        # Assemble layout
        self.app.layout = dbc.Container([
            header,

            dbc.Row([
                dbc.Col([controls], width=3),
                dbc.Col([main_content], width=9)
            ], className="mt-4"),

            # Hidden data stores
            dcc.Store(id='current-data-store'),
            dcc.Store(id='market-data-store'),

            # Auto-refresh interval
            dcc.Interval(
                id='auto-refresh-interval',
                interval=300000,  # 5 minutes
                n_intervals=0
            )
        ], fluid=True, style={'backgroundColor': '#1a1a1a'})

    def _setup_callbacks(self):
        """Setup dashboard callbacks"""

        @self.app.callback(
            [Output('market-dropdown', 'options'),
             Output('market-dropdown', 'value')],
            Input('asset-class-dropdown', 'value')
        )
        def update_market_options(asset_class):
            config = Config()
            markets = list(config.ASSET_CLASSES.get(asset_class, {}).keys())
            options = [{'label': m, 'value': m} for m in markets]
            return options, markets[0] if markets else None

        @self.app.callback(
            [Output('ticker-dropdown', 'options'),
             Output('ticker-dropdown', 'value')],
            [Input('asset-class-dropdown', 'value'),
             Input('market-dropdown', 'value')]
        )
        def update_ticker_options(asset_class, market):
            config = Config()
            tickers = config.ASSET_CLASSES.get(asset_class, {}).get(market, [])
            options = [{'label': t, 'value': t} for t in tickers]
            return options, tickers[0] if tickers else None

        @self.app.callback(
            [Output('current-data-store', 'data'),
             Output('sentiment-gauge-container', 'children'),
             Output('price-chart-container', 'children'),
             Output('sentiment-trend-container', 'children'),
             Output('sentiment-distribution-container', 'children'),
             Output('news-table-container', 'children'),
             Output('quick-stats', 'children')],
            [Input('analyze-button', 'n_clicks'),
             Input('refresh-button', 'n_clicks'),
             Input('auto-refresh-interval', 'n_intervals')],
            [State('ticker-dropdown', 'value')]
        )
        def update_dashboard(analyze_clicks, refresh_clicks, interval_clicks, ticker):
            if not ticker:
                return [None] + [html.Div("Please select a ticker to analyze")] * 6

            # Analyze the asset
            data = self.analyzer.analyze_asset(ticker)
            if not data:
                return [None] + [html.Div(f"Error analyzing {ticker}")] * 6

            sentiment = data['sentiment']
            price_data = data['price_data']

            # 1. Sentiment Gauge
            gauge_fig = self.analyzer.visualizer.create_sentiment_gauge(
                sentiment['overall_score'],
                sentiment['sentiment_label'],
                ticker
            )
            gauge = dcc.Graph(figure=gauge_fig, config={'displayModeBar': False})

            # 2. Price Chart
            price_fig = self.analyzer.visualizer.create_price_chart(price_data, ticker)
            price_chart = dcc.Graph(figure=price_fig)

            # 3. Sentiment Trend
            history = self.analyzer.get_sentiment_history(ticker, days=30)
            trend_fig = self.analyzer.visualizer.create_sentiment_trend(history)
            trend_chart = dcc.Graph(figure=trend_fig)

            # 4. Sentiment Distribution
            dist_fig = self.analyzer.visualizer.create_sentiment_distribution(sentiment)
            dist_chart = dcc.Graph(figure=dist_fig)

            # 5. News Table
            news_df = self.analyzer.get_recent_news(ticker, limit=10)
            news_table = self._create_news_table(news_df)

            # 6. Quick Stats
            quick_stats = self._create_quick_stats(data)

            return [data, gauge, price_chart, trend_chart, dist_chart, news_table, quick_stats]

        @self.app.callback(
            Output('market-analysis-results', 'children'),
            Input('market-analysis-button', 'n_clicks'),
            [State('asset-class-dropdown', 'value'),
             State('market-dropdown', 'value')]
        )
        def run_market_analysis(n_clicks, asset_class, market):
            if n_clicks == 0:
                return html.Div("Click the button to analyze the market")

            config = Config()
            tickers = config.ASSET_CLASSES.get(asset_class, {}).get(market, [])

            if not tickers:
                return html.Div("No tickers found for this market")

            # Limit to 5 tickers for performance
            tickers = tickers[:5]

            progress = html.Div([
                html.P(f"Analyzing {len(tickers)} assets in {market}..."),
                dbc.Progress(id="progress", value=0, max=100, className="mb-3")
            ])

            # Run batch analysis
            results_df = self.analyzer.batch_analyze(tickers)

            if results_df.empty:
                return html.Div("No results available")

            # Create market overview
            market_fig = self._create_market_overview_chart(results_df)
            market_chart = dcc.Graph(figure=market_fig)

            # Generate report
            report = self.analyzer.generate_report(results_df)
            report_display = html.Div([
                html.H5("Market Analysis Report", className="mt-4"),
                html.Pre(report, style={
                    'whiteSpace': 'pre-wrap',
                    'wordBreak': 'break-word',
                    'backgroundColor': '#2c2c2c',
                    'padding': '15px',
                    'borderRadius': '5px',
                    'fontSize': '14px'
                })
            ])

            return html.Div([market_chart, report_display])

    def _create_news_table(self, news_df):
        """Create news table component"""
        if news_df.empty:
            return html.Div("No recent news found")

        rows = []
        for _, row in news_df.iterrows():
            sentiment_color = Config.SENTIMENT_COLORS.get(
                row.get('sentiment_label', 'Neutral'),
                '#CCCCCC'
            )

            rows.append(
                dbc.Card([
                    dbc.CardBody([
                        html.H6(
                            row['title'][:100] + "..." if len(row['title']) > 100 else row['title'],
                            className="card-title"
                        ),
                        html.P(
                            row['source'],
                            className="card-subtitle mb-2 text-muted"
                        ),
                        html.Span(
                            f"Sentiment: {row.get('sentiment_label', 'Neutral')} "
                            f"({row.get('sentiment_score', 0):.2f})",
                            style={
                                'color': sentiment_color,
                                'fontWeight': 'bold'
                            },
                            className="mb-2 d-block"
                        ),
                        html.A(
                            "Read Full Article →",
                            href=row['url'],
                            target="_blank",
                            className="btn btn-sm btn-outline-primary"
                        )
                    ])
                ], className="mb-3")
            )

        return html.Div(rows)

    def _create_quick_stats(self, data):
        """Create quick stats display"""
        if not data:
            return html.Div("No data available")

        sentiment = data['sentiment']

        stats = [
            html.P(f"📊 Sources Analyzed: {sentiment['sources_analyzed']}"),
            html.P(f"✅ Positive Sources: {sentiment['positive_count']}"),
            html.P(f"🔄 Neutral Sources: {sentiment['neutral_count']}"),
            html.P(f"❌ Negative Sources: {sentiment['negative_count']}"),
            html.P(f"🎯 Confidence: {sentiment['confidence']:.2f}"),
        ]

        if data.get('current_price'):
            stats.append(html.P(f"💰 Current Price: ${data['current_price']:.2f}"))

        return html.Div(stats)

    def _create_market_overview_chart(self, results_df):
        """Create market overview chart"""
        fig = go.Figure()

        # Add bars for sentiment score
        fig.add_trace(go.Bar(
            x=results_df['ticker'],
            y=results_df['sentiment_score'],
            name='Sentiment Score',
            marker_color=['#00AA00' if x > 0 else '#FF0000' for x in results_df['sentiment_score']],
            text=results_df['sentiment_score'].round(3),
            textposition='auto'
        ))

        fig.update_layout(
            title=f"Market Sentiment Overview ({len(results_df)} assets)",
            height=400,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font={'color': "white", 'family': "Arial"},
            xaxis_title="Ticker",
            yaxis_title="Sentiment Score",
            showlegend=False
        )

        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')

        return fig

    def run(self, port=8050, mode='inline'):
        """Run the dashboard"""
        print(f"🚀 Starting dashboard on port {port}...")
        print("📊 Dashboard will open in a new window")
        print("🔄 Close the dashboard with Ctrl+C in the terminal")

        if globals().get('IN_COLAB', False):
            try:
                from pyngrok import ngrok
                ngrok.kill() # Ensure previous tunnels are closed
                public_url = ngrok.connect(port).public_url
                print(f"🔗 Your dashboard is available at: {public_url}")
                self.app.run(
                    port=port,
                    debug=False,
                    jupyter_mode='external' # Use external mode for ngrok
                )
            except Exception as ngrok_e:
                print(f"❌ Error setting up ngrok or running in external mode: {ngrok_e}")
                print("Trying to run dashboard in internal mode, port forwarding might be required.")
                self.app.run(
                    port=port,
                    debug=False,
                    # Fallback to inline mode if ngrok fails
                    jupyter_mode='inline'
                )
        else:
            # For local execution
            self.app.run(
                jupyter_mode=mode,
                port=port,
                debug=False,
                dev_tools_ui=False
            )

In [10]:
# ============================================================================
# MAIN EXECUTION BLOCK
# ============================================================================

def main():
    """Main function to run the application"""
    print("\n" + "="*70)
    print("💰 FINANCIAL SENTIMENT ANALYZER - SINGLE FILE EDITION")
    print("="*70)
    print("\nInitializing system...")

    # Initialize the analyzer
    analyzer = FinancialSentimentAnalyzer()

    while True:
        # Ask user what they want to do
        print("\n" + "="*70)
        print("MENU OPTIONS:")
        print("1. Launch Interactive Dashboard (Recommended)")
        print("2. Analyze Single Asset")
        print("3. Batch Analyze Market")
        print("4. Generate Market Report")
        print("5. Exit")
        print("="*70)

        choice = input("\nEnter your choice (1-5): ").strip()

        if choice == "1":
            # Launch dashboard
            print("\n🚀 Launching Interactive Dashboard...")
            print("📊 Please wait for dashboard to load...")
            print("🔗 If running in Colab, look for the public URL above")

            dashboard = FinancialDashboard(analyzer)
            try:
                # For Colab, use 'inline' or 'jupyterlab' mode
                dashboard.run(port=8050, mode='inline')
            except Exception as e:
                print(f"Error: {e}")
                print("Trying alternative port...")
                dashboard.run(port=random.randint(8000, 9000), mode='inline')

        elif choice == "2":
            # Analyze single asset
            ticker = input("Enter ticker symbol (e.g., AAPL, BTC-USD): ").strip().upper()
            if ticker:
                result = analyzer.analyze_asset(ticker)
                if result:
                    print("\n" + "="*70)
                    print(f"ANALYSIS RESULTS FOR {ticker}")
                    print("="*70)
                    print(f"Sentiment Score: {result['sentiment']['overall_score']:.3f}")
                    print(f"Sentiment Label: {result['sentiment']['sentiment_label']}")
                    print(f"Sources Analyzed: {result['sentiment']['sources_analyzed']}")
                    if result.get('current_price'):
                        print(f"Current Price: ${result['current_price']:.2f}")

        elif choice == "3":
            # Batch analyze
            asset_class = input("Enter asset class (equity/commodities/forex/crypto): ").strip().lower()
            if asset_class in Config.ASSET_CLASSES:
                markets = list(Config.ASSET_CLASSES[asset_class].keys())
                print(f"Available markets: {', '.join(markets)}")
                market = input(f"Select market ({'/'.join(markets)}): ").strip().upper()

                if market in markets:
                    tickers = Config.ASSET_CLASSES[asset_class][market]
                    print(f"Analyzing {len(tickers)} assets in {market}...")
                    results = analyzer.batch_analyze(tickers)
                    print(analyzer.generate_report(results))
                else:
                    print("Invalid market selection.")
            else:
                print("Invalid asset class selection.")

        elif choice == "4":
            # Generate market report
            results = analyzer.batch_analyze()
            print("\n" + "="*70)
            print("MARKET SENTIMENT REPORT")
            print("="*70)
            print(analyzer.generate_report(results))

        elif choice == "5":
            print("Exiting... Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")
            continue

        # Ask if user wants to continue
        print("\n" + "="*70)
        continue_choice = input("Would you like to run another analysis? (y/n): ").strip().lower()
        if continue_choice != 'y':
            print("Thank you for using the Financial Sentiment Analyzer!")
            break

# ============================================================================
# DEMO MODE (For quick testing)
# ============================================================================

def run_demo():
    """Run a quick demo of the system"""
    print("\n🎮 DEMO MODE: Testing with AAPL...")

    # Initialize
    analyzer = FinancialSentimentAnalyzer()

    # Analyze Apple
    result = analyzer.analyze_asset("AAPL")

    if result:
        print(f"\n✓ Analysis complete!")
        print(f"  Sentiment: {result['sentiment']['sentiment_label']}")
        print(f"  Score: {result['sentiment']['overall_score']:.3f}")
        print(f"  Sources: {result['sentiment']['sources_analyzed']}")

        # Show sample news
        if result['news']:
            print(f"\n📰 Sample News Headlines:")
            for i, article in enumerate(result['news'][:3], 1):
                print(f"  {i}. {article['title'][:60]}...")

        # Run batch analysis
        print(f"\n📊 Running market analysis...")
        results_df = analyzer.batch_analyze(['AAPL', 'GOOGL', 'MSFT', 'TSLA'])
        print(analyzer.generate_report(results_df))

        print("\n🎉 Demo complete! Run main() for full functionality.")

# ============================================================================
# COLAB-SPECIFIC SETUP
# ============================================================================

def setup_colab():
    """Setup for Google Colab environment"""
    print("🔧 Setting up Google Colab environment...")

    # Install ngrok for tunneling
    !pip install pyngrok -q

    # Set environment variables
    import os
    os.environ['DISABLE_JUPYTER_DASH_TUNNEL'] = '1'

    print("✓ Colab setup complete!")
    print("\nTo use the dashboard in Colab:\n1. Run main() and choose option 1\n2. Look for the output with a URL like: http://localhost:8050\n3. Use ngrok or Colab's built-in port forwarding")

    return True

# ============================================================================
# ENTRY POINT
# ============================================================================

def start_cli():
    """Entry point for interactive usage in scripts or notebooks"""
    print("\n" + "="*70)
    print("WELCOME TO FINANCIAL SENTIMENT ANALYZER")
    print("="*70)
    print("\nA professional-grade sentiment analysis system for:")
    print("• Equity Markets (US, UK, India)")
    print("• Commodities (Metals, Energy, Agriculture)")
    print("• Forex (Majors, Minors, Exotics)")
    print("• Cryptocurrency (Bitcoin, Ethereum, Altcoins)")

    # Check if running in Colab
    try:
        import google.colab
        IN_COLAB = True
        print("\n📱 Running in Google Colab environment")
        setup_colab()
    except:
        IN_COLAB = False
        print("\n💻 Running in local Python environment")

    # Ask user what to do
    while True:
        print("\nSelect an option:")
        print("1. Run full application")
        print("2. Run quick demo")
        print("3. Exit")

        option = input("\nEnter choice (1-3): ").strip()

        if option == "1":
            main()
        elif option == "2":
            run_demo()
        elif option == "3":
            print("Goodbye!")
            break
        else:
            print("Invalid option. Please try again.")



In [ ]:
if __name__ == "__main__" or 'ipykernel' in sys.modules:
    start_cli()


WELCOME TO FINANCIAL SENTIMENT ANALYZER

A professional-grade sentiment analysis system for:
• Equity Markets (US, UK, India)
• Commodities (Metals, Energy, Agriculture)
• Forex (Majors, Minors, Exotics)
• Cryptocurrency (Bitcoin, Ethereum, Altcoins)

💻 Running in local Python environment

Select an option:
1. Run full application
2. Run quick demo
3. Exit



Enter choice (1-3):  1



💰 FINANCIAL SENTIMENT ANALYZER - SINGLE FILE EDITION

Initializing system...


Loading weights: 100%|████████████████| 201/201 [00:00<00:00, 331.10it/s, Materializing param=classifier.weight]
BertForSequenceClassification LOAD REPORT from: ProsusAI/finbert
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✓ FinBERT model loaded successfully
✓ Financial Sentiment Analyzer initialized
✓ Tracking 50 assets across 4 markets

MENU OPTIONS:
1. Launch Interactive Dashboard (Recommended)
2. Analyze Single Asset
3. Batch Analyze Market
4. Generate Market Report
5. Exit


...

✓ Analysis complete for BZ=F
  Sentiment: Neutral (0.058)
  Sources analyzed: 40
  Current price: $450.58


$SI=F: possibly delisted; no price data found  (period=1d)


🔍 Analyzing SI=F...


[2026-02-22 17:38:18,878] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "d:\self made projects\Sentiment_analysis\.venv-2\lib\site-packages\dash\_callback.py", line 738, in add_context
    jsonResponse = to_json(response)
  File "d:\self made projects\Sentiment_analysis\.venv-2\lib\site-packages\dash\_utils.py", line 28, in to_json
    return to_json_plotly(value)
  File "d:\self made projects\Sentiment_analysis\.venv-2\lib\site-packages\plotly\io\_json.py", line 143, in to_json_plotly
    json.dumps(plotly_object, cls=PlotlyJSONEncoder, **opts), _swap_json
  File "C:\Users\Kishan\AppData\Local\Programs\Python\Python310\lib\json\__init__.py", line 238, in dumps
    **kw).encode(obj)
  File "d:\self made projects\Sentiment_analysis\.venv-2\lib\site-packages\_plotly_utils\utils.py", line 162, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\Kishan\AppData\Local\Programs\Python\Python310\lib\json\enco

✓ Analysis complete for SI=F
  Sentiment: Positive (0.278)
  Sources analyzed: 40
  Current price: $462.20
